In [1]:
from pyspark import SparkContext
from datetime import datetime
import random

In [2]:
sc = SparkContext(appName="Movie plots")

In [3]:
metadata = sc.textFile("movie.metadata.tsv")

In [4]:
k_gram_size=10

In [5]:
 metadata = metadata.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[2]))

In [6]:
metadata.take(10)

[(975900, 'Ghosts of Mars'),
 (3196793, 'Getting Away with Murder: The JonBenét Ramsey Mystery'),
 (28463795, 'Brun bitter'),
 (9363483, 'White Of The Eye'),
 (261236, 'A Woman in Flames'),
 (13696889, 'The Gangsters'),
 (18998739, "The Sorcerer's Apprentice"),
 (10408933, "Alexander's Ragtime Band"),
 (9997961, 'Contigo y aquí'),
 (2345652, 'City of the Dead')]

In [7]:
#movie_plots = sc.textFile("plot_summaries.txt")
movie_plots = sc.textFile("plot_summaries.txt")

In [8]:
movie_plots = movie_plots.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[1]))
movie_plots = movie_plots.sample(withReplacement=False, fraction=0.2)

In [9]:
movie_plots.take(1)

[(24225279,
  "The story begins with Hannah, a young Jewish teen, as she is completing her senior year of high school. Her small neighborhood in Brooklyn is falling apart and SING! is one of the only traditions keeping the neighborhood alive. Newly arrived teacher, Miss Lombardo grew up in the neighborhood but returned to be their Sing Leader. One cold Christmas night, Miss Lombardo is leaving a neighborhood party when a young man hails her a cab, then attempts to mug her. In self defense, she bites his hand to release his grip and he screams in pain and terror, quickly making an exit. The cab driver jokes about not starting the meter yet. On the first day of school, Miss Lombardo runs into difficulty when her students are uninterested and misbehaved. One such student was named Dominic who gets scolded for bringing stolen watches to school grounds and putting his feet up on the desk. On the day of Sing Leader elections, Miss Lombardo recognizes Dominic as her mugger by the bandage on h

In [10]:
#shingle_set = {}


In [11]:
def break_k_grams(plot):
    shingles = []
    for i in range(len(plot)-k_gram_size+1):
        shingles.append(plot[i:i+k_gram_size])
        
    return shingles

In [12]:
movie_shingles = movie_plots.mapValues(break_k_grams)

In [13]:
movie_shingles.take(1)


[(24225279,
  ['The story ',
   'he story b',
   'e story be',
   ' story beg',
   'story begi',
   'tory begin',
   'ory begins',
   'ry begins ',
   'y begins w',
   ' begins wi',
   'begins wit',
   'egins with',
   'gins with ',
   'ins with H',
   'ns with Ha',
   's with Han',
   ' with Hann',
   'with Hanna',
   'ith Hannah',
   'th Hannah,',
   'h Hannah, ',
   ' Hannah, a',
   'Hannah, a ',
   'annah, a y',
   'nnah, a yo',
   'nah, a you',
   'ah, a youn',
   'h, a young',
   ', a young ',
   ' a young J',
   'a young Je',
   ' young Jew',
   'young Jewi',
   'oung Jewis',
   'ung Jewish',
   'ng Jewish ',
   'g Jewish t',
   ' Jewish te',
   'Jewish tee',
   'ewish teen',
   'wish teen,',
   'ish teen, ',
   'sh teen, a',
   'h teen, as',
   ' teen, as ',
   'teen, as s',
   'een, as sh',
   'en, as she',
   'n, as she ',
   ', as she i',
   ' as she is',
   'as she is ',
   's she is c',
   ' she is co',
   'she is com',
   'he is comp',
   'e is compl',
   ' is comple',
  

In [14]:
#optional but advised hashing of shingles into 4 bytes (%2**4*8)
movie_shingles = movie_shingles.mapValues(lambda shing: sorted([hash(x)%(2**32) for x in shing]))
#movie_shingles = movie_shingles.mapValues(lambda shing: sorted([x for x in shing]))

movie_shingles.take(2)

[(24225279,
  [1167070,
   1533041,
   1627370,
   2166765,
   3451129,
   3984207,
   5245461,
   5623064,
   5823642,
   7050450,
   9918269,
   12133824,
   14775446,
   15858891,
   18437153,
   19155341,
   21350329,
   21750935,
   22187724,
   22904957,
   23731673,
   26588629,
   26743377,
   28518539,
   29850957,
   31195610,
   32107288,
   32716379,
   33152140,
   34213004,
   34256329,
   34580088,
   38759977,
   40401508,
   41470657,
   44718738,
   46682771,
   47138309,
   47759796,
   49184182,
   49764568,
   49901227,
   50226158,
   50667035,
   52579774,
   54342551,
   56542764,
   58887308,
   60242691,
   60257003,
   61045499,
   66313379,
   67096545,
   67671276,
   68885265,
   76157287,
   76240299,
   76240299,
   77580401,
   80967155,
   81345685,
   83118793,
   83533130,
   83549309,
   83654617,
   83841314,
   84285189,
   84285189,
   84285189,
   84285189,
   84285189,
   85102992,
   86837942,
   86986965,
   88735451,
   88735451,
   89249537

In [15]:
def random_hash():
    a = random.randint(1,1000)
    b = random.randint(0,1000)
    return a,b

In [16]:
random_hash()

(688, 138)

In [17]:
#n_shingles = 23254807 
n_shingles = movie_shingles.flatMap(lambda tup: tup[1]).distinct().count()

In [18]:
#prime number for hashing
p = 24000001
#number of hash functions
k=100
hash_functions = []
for i in range(k):
    hash_functions.append(random_hash())
hash_functions

[(952, 164),
 (267, 407),
 (243, 554),
 (166, 884),
 (342, 688),
 (297, 213),
 (752, 264),
 (881, 205),
 (186, 434),
 (655, 916),
 (279, 625),
 (269, 343),
 (323, 476),
 (214, 299),
 (8, 634),
 (422, 245),
 (578, 533),
 (809, 407),
 (791, 221),
 (250, 388),
 (129, 885),
 (352, 451),
 (136, 437),
 (158, 870),
 (603, 319),
 (830, 446),
 (169, 60),
 (55, 820),
 (127, 29),
 (489, 812),
 (288, 411),
 (404, 91),
 (685, 406),
 (994, 599),
 (359, 355),
 (40, 757),
 (48, 630),
 (3, 387),
 (100, 0),
 (558, 479),
 (256, 143),
 (394, 261),
 (136, 298),
 (190, 955),
 (795, 533),
 (910, 393),
 (506, 756),
 (473, 499),
 (355, 352),
 (571, 407),
 (440, 213),
 (456, 750),
 (106, 364),
 (747, 10),
 (827, 198),
 (58, 314),
 (990, 125),
 (757, 954),
 (569, 275),
 (776, 378),
 (124, 774),
 (830, 60),
 (484, 242),
 (681, 931),
 (118, 842),
 (354, 667),
 (169, 925),
 (94, 959),
 (145, 583),
 (155, 340),
 (232, 824),
 (995, 604),
 (170, 927),
 (512, 172),
 (375, 653),
 (870, 893),
 (162, 158),
 (705, 134),
 (

In [19]:
def get_signature(hashed_shingles):
    signature_matrix = {}
    for element in hashed_shingles:
        for i in range(k):
            hash_result = calc_hash(*hash_functions[i],p,n_shingles, element)
            if  hash_result < signature_matrix.get(i,float('inf')):
                signature_matrix[i]=hash_result
    return signature_matrix

In [20]:
def calc_hash(a,b,prime,n,value):
    return ((a*value + b)%prime)%n
    

In [21]:
movie_signatures = movie_shingles.mapValues(get_signature)
movie_signatures.take(2)

[(24225279,
  {0: 3520,
   1: 1312,
   2: 2255,
   3: 2046,
   4: 3082,
   5: 1360,
   6: 989,
   7: 6064,
   8: 1736,
   9: 1489,
   10: 2578,
   11: 1984,
   12: 2737,
   13: 820,
   14: 124,
   15: 1094,
   16: 1167,
   17: 571,
   18: 2452,
   19: 1751,
   20: 1788,
   21: 333,
   22: 1327,
   23: 1251,
   24: 3946,
   25: 4308,
   26: 1243,
   27: 332,
   28: 918,
   29: 1505,
   30: 1964,
   31: 111,
   32: 4760,
   33: 2263,
   34: 248,
   35: 760,
   36: 966,
   37: 105,
   38: 313,
   39: 1032,
   40: 356,
   41: 2966,
   42: 1188,
   43: 187,
   44: 2188,
   45: 1723,
   46: 433,
   47: 3810,
   48: 2385,
   49: 1232,
   50: 581,
   51: 1049,
   52: 254,
   53: 2714,
   54: 487,
   55: 720,
   56: 424,
   57: 6253,
   58: 356,
   59: 5810,
   60: 322,
   61: 3922,
   62: 2269,
   63: 145,
   64: 95,
   65: 166,
   66: 2108,
   67: 315,
   68: 265,
   69: 1425,
   70: 1811,
   71: 1787,
   72: 2117,
   73: 584,
   74: 1669,
   75: 258,
   76: 791,
   77: 1211,
   78: 6410,
   

In [22]:
movie_signatures.cache()

PythonRDD[14] at RDD at PythonRDD.scala:53

In [23]:
n_bands = 20
n_rows = 5
similarity_threshold = (1/n_bands)**(1/n_rows)
similarity_threshold

0.5492802716530588

In [24]:
def jaccard_similarity(set_a,set_b):
    intersection = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))
    return intersection/union

In [25]:
jaccard_similarity({1,2,3},{3,4,5})

0.2

In [26]:
n_lsh_buckets = 1024
lsh_buckets={}


In [27]:
def break_shingles_to_bands(shingles):
    set_list = []
    for _ in range(n_bands):
        set_list.append(set())
        
    for i in shingles:
        set_list[i//n_rows].add(shingles[i])
    return set_list
    

In [28]:
test = {0: 1179,
   1: 138226,
   2: 58526,
   3: 50635,
   4: 63542,
   5: 18022,
   6: 142334,
   7: 90061,
   8: 20921,
   9: 187417,
   10: 66231,
   11: 149743,
   12: 55278,
   13: 16198,
   14: 29504,
   15: 137872,
   16: 26262,
   17: 121676,
   18: 8240,
   19: 55028,
   20: 32945,
   21: 191234,
   22: 61885,
   23: 58198,
   24: 185256,
   25: 21165,
   26: 407631,
   27: 3969,
   28: 26739,
   29: 16283,
   30: 121042,
   31: 21630,
   32: 59449,
   33: 11024,
   34: 91822,
   35: 209568,
   36: 158652,
   37: 87375,
   38: 39936,
   39: 38781,
   40: 31799,
   41: 33956,
   42: 117648,
   43: 121603,
   44: 115801,
   45: 7263,
   46: 25610,
   47: 15775,
   48: 102872,
   49: 99004,
   50: 168497,
   51: 93079,
   52: 192358,
   53: 10220,
   54: 14705,
   55: 45435,
   56: 8949,
   57: 8655,
   58: 58414,
   59: 25409,
   60: 44614,
   61: 143825,
   62: 39789,
   63: 2768,
   64: 164976,
   65: 34187,
   66: 3309,
   67: 1087,
   68: 103689,
   69: 60480,
   70: 129943,
   71: 10567,
   72: 43456,
   73: 6431,
   74: 60255,
   75: 15198,
   76: 5965,
   77: 58259,
   78: 140677,
   79: 36962,
   80: 15882,
   81: 59376,
   82: 146360,
   83: 6505,
   84: 3238,
   85: 64314,
   86: 96034,
   87: 96831,
   88: 138769,
   89: 172868,
   90: 150154,
   91: 115317,
   92: 27755,
   93: 186290,
   94: 6132,
   95: 2436,
   96: 23769,
   97: 84076,
   98: 117144,
   99: 76668}
break_shingles_to_bands(test)

[{1179, 50635, 58526, 63542, 138226},
 {18022, 20921, 90061, 142334, 187417},
 {16198, 29504, 55278, 66231, 149743},
 {8240, 26262, 55028, 121676, 137872},
 {32945, 58198, 61885, 185256, 191234},
 {3969, 16283, 21165, 26739, 407631},
 {11024, 21630, 59449, 91822, 121042},
 {38781, 39936, 87375, 158652, 209568},
 {31799, 33956, 115801, 117648, 121603},
 {7263, 15775, 25610, 99004, 102872},
 {10220, 14705, 93079, 168497, 192358},
 {8655, 8949, 25409, 45435, 58414},
 {2768, 39789, 44614, 143825, 164976},
 {1087, 3309, 34187, 60480, 103689},
 {6431, 10567, 43456, 60255, 129943},
 {5965, 15198, 36962, 58259, 140677},
 {3238, 6505, 15882, 59376, 146360},
 {64314, 96034, 96831, 138769, 172868},
 {6132, 27755, 115317, 150154, 186290},
 {2436, 23769, 76668, 84076, 117144}]

In [29]:
def hash_bands(doc_band_set):
    bucket_docs = []
    for s in doc_band_set[1]:
        
        bucket_index = hash(str(s))%n_lsh_buckets
        bucket_docs.append((bucket_index,doc_band_set[0]))
        
    return bucket_docs

In [30]:
hash_band_rdd = movie_signatures.mapValues(break_shingles_to_bands).flatMap(hash_bands)
hash_band_rdd.take(8)
#candidate_proximity_rdd.collect()

[(933, 24225279),
 (167, 24225279),
 (834, 24225279),
 (357, 24225279),
 (575, 24225279),
 (412, 24225279),
 (150, 24225279),
 (830, 24225279)]

In [31]:
candidate_proximity_rdd = hash_band_rdd.groupByKey()


In [32]:
#####INTERRUPTED BECAUSE IT NEVER ENDED
candidate_proximity_rdd.take(10)


[(933, <pyspark.resultiterable.ResultIterable at 0x7f316ffd15b0>),
 (834, <pyspark.resultiterable.ResultIterable at 0x7f316ffd1760>),
 (357, <pyspark.resultiterable.ResultIterable at 0x7f316ffd19d0>),
 (150, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f190>),
 (162, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f040>),
 (846, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f7f0>),
 (324, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f070>),
 (258, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f1c0>),
 (651, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f250>),
 (519, <pyspark.resultiterable.ResultIterable at 0x7f316ff0f310>)]

In [33]:
def compare_pairs(it):
    
    
    candidate_list = [x for x in iter(it)]
    pairs = []
    for i in range(len(candidate_list)):
        for j in range(i+1,len(candidate_list)):
            pairs.append((candidate_list[i],candidate_list[j]))
    
    return pairs

In [34]:
candidate_proximity_rdd = candidate_proximity_rdd.mapValues(compare_pairs)

In [35]:
def compare_signatures(doc1,doc2):
    #signatures_1 = movie_signatures.lookup(pair[0])
    #signatures_2 = movie_signatures.lookup(pair[1])
    signatures_1 = set(sig_dict.value[doc1].values())
    signatures_2 = set(sig_dict.value[doc2].values())
    #return doc1
    return jaccard_similarity(signatures_1,signatures_2)

In [36]:
candidate_proximity_rdd.take(2)

[(933,
  [(24225279, 13448042),
   (24225279, 16673820),
   (24225279, 8896753),
   (24225279, 31976092),
   (24225279, 32691963),
   (24225279, 5844491),
   (24225279, 4251465),
   (24225279, 5623405),
   (24225279, 29019202),
   (24225279, 35618756),
   (24225279, 28402972),
   (24225279, 26138411),
   (24225279, 1710716),
   (24225279, 24381557),
   (24225279, 27499456),
   (24225279, 4148842),
   (24225279, 3159738),
   (24225279, 6873331),
   (24225279, 4829230),
   (24225279, 8590427),
   (24225279, 11220896),
   (24225279, 144813),
   (24225279, 14503106),
   (24225279, 20239110),
   (24225279, 7027409),
   (24225279, 9970689),
   (24225279, 27604230),
   (24225279, 25714957),
   (24225279, 2038261),
   (24225279, 4598013),
   (24225279, 6640066),
   (24225279, 61507),
   (24225279, 8934867),
   (24225279, 28796274),
   (24225279, 18640429),
   (24225279, 11043663),
   (24225279, 27265213),
   (24225279, 318542),
   (24225279, 20570407),
   (24225279, 10378192),
   (24225279, 27

In [37]:
sig_dict = sc.broadcast(movie_signatures.collectAsMap())

real_pairs = candidate_proximity_rdd.flatMap(lambda candidates: [(c,compare_signatures(*c)) for c in candidates[1]]).filter(lambda a: a[1]>=similarity_threshold and a[1]<0.98)
real_pairs.take(10)

[((10134112, 7082486), 0.5503875968992248),
 ((18544488, 19876978), 0.5748031496062992),
 ((10134112, 7082486), 0.5503875968992248),
 ((18544488, 19876978), 0.5748031496062992),
 ((18544488, 19876978), 0.5748031496062992),
 ((10134112, 7082486), 0.5503875968992248),
 ((18544488, 19876978), 0.5748031496062992)]

In [46]:
shingles_dict = sc.broadcast(movie_shingles.collectAsMap())

In [49]:
def exact_comparison(docid_1,docid_2):
    set_1 = set(shingles_dict.value[docid_1])
    set_2 = set(shingles_dict.value[docid_2])
    return jaccard_similarity(set_1,set_2)

In [50]:
#example
exact_comparison(18544488,19876978)

0.7918939984411536

In [ ]:
####REMOVE/FILTER DUPLICATES

In [56]:
def given_a_movie(movie_id):
    similar_movies = real_pairs.filter(lambda a: movie_id in list(a[0]) and 0.98>=a[1]>=0.8)
    return similar_movies.collect()